#### Branch and Bound
João Antonio Honorato, matrícula: 20210026680

Maria Raquel Martinez, matrícula: 20200025900

In [1]:
from mip import (CBC, CONTINUOUS, MAXIMIZE, Model, xsum, OptimizationStatus)

In [2]:
## A função cria_problema lê o arquivo txt com os dados de entrada
## e os traduz para um modelo do tipo Model
## recebe duas strings, o caminho para o arquivo e um nome para o problema (opcional)
def cria_problema(caminho, nome = ""):

        ## lendo o arquivo:

    entrada = open(caminho, "r") 

    linhas = entrada.readlines()
    # lista de strings, cada elemento é uma linha da entrada

    n_var = int(linhas[0].split()[0]) # número de variáveis
    n_restr = int(linhas[0].split()[1]) # número de restrições

    entrada.close()

        ## instanciando o problema:

    modelo = Model(nome, sense = MAXIMIZE, solver_name = CBC)

    modelo.verbose = 0
    # desabilitando o printzão enorme

    x = {i: modelo.add_var(var_type = CONTINUOUS, name = f'x{i+1}', lb = 0.0, ub = 1.0,) for i in range(n_var)}

    funcao_objetivo = [int(a) for a in linhas[1].split()]
    # criando uma lista de inteiros com os coeficientes das variáveis na função objetivo

    modelo.objective = xsum(funcao_objetivo[i] * x[i] for i in range(n_var))
    # adicionando a função objetivo ao modelo

    for i in range(n_restr):
        coeficientes = [int(a) for a in linhas[i+2].split()]
        # criando uma lista de inteiros que representa a restrição
        # os (n_var - 1) primeiros números são os coeficientes de cada variável (para o LHS)
        # o último número da lista é o RHS da restrição
        modelo += xsum(coeficientes[j] * x[j] for j in range(n_var)) <= coeficientes[-1]
        # adicionando a restrição ao modelo
    
    status = modelo.optimize()
    # rodando o simplex

    return modelo

In [11]:
## imprime as informações da solução
def imprime_solucao(modelo):
  print("Problema: " + modelo.name)
  print("Status: ", modelo.status)
  
  if modelo.status == OptimizationStatus.INFEASIBLE:
    return  
  if modelo.status == OptimizationStatus.NO_SOLUTION_FOUND:
    return  
  print(f"Valor da solução: {modelo.objective_value:.2f}")
  
  print("Variáveis:")
  for v in modelo.vars:
      print(f"  {v.name} = {v.x:.2f}")

In [4]:
## imprime nome, objetivo, função objetivo e restrições
def imprime_modelo(model):
  model.write("model.lp") # salva modelo em arquivo
  with open("model.lp", "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())
  
  # fazemos assim porque write e open são métodos próprios da classe Model
  # porém não tem um método para imprimí-lo sem essa gambiarra

In [5]:
## retorna True se todas as variáveis da solução do modelo assumem valores inteiros
def poda_por_integralidade(modelo : Model):
    for v in modelo.vars: # percorre as variáveis
      if v.x % 1 != 0: # calcula o resto da divisão do valor por 1
        return False # se uma for diferente de 0, é porque seu valor é fracionário
    
    return True # se o loop foi encerrado sem retornar False, todas as variáveis são inteiras

In [ ]:
## retorna True se a solução encontrada é inviável ou inexistente
def poda_por_inviabilidade(modelo : Model):
    if modelo.status == OptimizationStatus.NO_SOLUTION_FOUND:
        return True
    if modelo.status == OptimizationStatus.INFEASIBLE:
        return True
    else:
        return False

In [ ]:
## retorna True se a solução encontrada é menor que o limite primal
def poda_por_limitante(modelo: Model, limite):
    if modelo.objective_value < limite:
        return True
    else:
        return False

In [6]:
## função que determina a variável na qual adicionaremos as restrições
## retorna dois novos modelos, com tais restrições
def ramifica(modelo : Model):

        ## escolhendo a variável a ser "restringida"

    i = 0 # contador, acompanha as variáveis percorridas
    variavel_indice = i
    menor_diferenca = 0.51
    # como as variáveis são todas 0 ou 1, a maior diferença sempre sera igual ou menor que 0.5

    for v in modelo.vars: 
        diferenca = abs(v.x - 0.5)
        if menor_diferenca > diferenca:
            menor_diferenca = diferenca
            variavel_indice = i
        i+=1
    # determinando a variável mais próxima de 0.5 e seu índice

        ## criando dois novos modelos, iguais ao anterior
        ## cada um com uma restrição nova, xi = 0 e xi = 1  

    modelo_novo_0 = modelo.copy()
    modelo_novo_1 = modelo.copy()
    # copiando

    modelo_novo_0 += modelo_novo_0.vars[variavel_indice] == 0
    modelo_novo_1 += modelo_novo_1.vars[variavel_indice] == 1
    # adicionando as restrições novas

    modelo_novo_0.verbose = 0
    modelo_novo_1.verbose = 0
    # desabilitando verbose (o printzão enorme)
    
    modelo_novo_0.optimize()
    modelo_novo_1.optimize()
    # rodando o simplex

    return modelo_novo_0, modelo_novo_1

In [7]:
def branch_and_bound(problema : Model):
    
    # temos uma fila de modelos a serem analisados (como os nós da árvore do branch and bound)
    # cada modelo vai ser processado e removido da fila 
    # ele pode ser podado ou ramificado
    # se ele é ramificado, ele enfileira mais dois modelos
    # tendo sido processado, ele é desenfileirado
    # o algoritmo continua enquanto ouverem modelos na fila

    fila = []
    fila.append(problema)

    melhor_solucao = float("-inf")

    while len(fila): # enquanto houverem modelos na fila
        
        if poda_por_inviabilidade(fila[0]):
            del fila[0]

        elif poda_por_integralidade(fila[0]):
            if fila[0].objective_value > melhor_solucao:
                melhor_modelo = fila[0]
                melhor_solucao = fila[0].objective_value
            del fila[0]

        elif poda_por_limitante(fila[0], melhor_solucao):
            del fila[0]

        else:
            modelo_0, modelo_1 = ramifica(fila[0])
            del fila[0]
            fila.append(modelo_0)
            fila.append(modelo_1)

    return melhor_modelo

In [16]:
modelo_teste = cria_problema("testes/teste0.txt", "TESTE 0")
modelo_1, modelo_2 = ramifica(modelo_teste)

model = modelo_1
imprime_solucao(model)

model = modelo_2
imprime_solucao(model)

Problema: TESTE 0
Status:  OptimizationStatus.OPTIMAL
Valor da solução: 11.75
Variáveis:
  x1 = 0.75
  x2 = 0.00
  x3 = 1.00
Problema: TESTE 0
Status:  OptimizationStatus.OPTIMAL
Valor da solução: 14.00
Variáveis:
  x1 = 0.00
  x2 = 1.00
  x3 = 0.50
